<a href="https://colab.research.google.com/github/kchaehyeon01/DLML/blob/main/Chapter7_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
# 4차원 배열
x = np.random.rand(10,1,28,28)
print("x.shape :", x.shape)
print("x[0].shape :", x[0].shape)
print("x[1].shape :", x[1].shape)
print(x[0][0].shape) # 첫 번째 데이터의 첫 채널의 공간 데이터에 접근

x.shape : (10, 1, 28, 28)
x[0].shape : (1, 28, 28)
x[1].shape : (1, 28, 28)
(28, 28)


# 7.4.3 합성곱 계층 구현하기

In [ ]:
# 교재 제공 함수 : https://github.com/WegraLee/deep-learning-from-scratch/blob/master/common/util.py

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    # 다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    print(col.shape)

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

# 합성곱 계층의 역전파 구현 (im2col을 역으로 처리)
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    # (im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

In [ ]:
x1 = np.random.rand(1,3,7,7) # datanum, channelnum, height, width
col1 = im2col(x1,5,5,stride=1,pad=0)
print(col1.shape)

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride=1,pad=0)
print(col2.shape)

(1, 3, 5, 5, 3, 3)
(9, 75)
(10, 3, 5, 5, 3, 3)
(90, 75)


In [ ]:
# im2col을 사용하여 합성곱 계층 구현
class Convolution:

  def __init__(self,W,b,stride=1,pad=0):
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad
    
  def forward(self,x):
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int(1+(H+2*self.pad-FH)/self.stride)
    out_w = int(1+(W+2*self.pad-FW)/self.stride)
    col = im2col(x,FH,FW,self.stride, self.pad)
    col_W = self.W.reshape(FN, -1).T # 필터 전개
    out = np.dot(col, col_W) + self.b
    out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
    return out

  def backward(self, dout):
    FN, C, FH, FW = self.W.shape
    dout = dout.transpose(0,2,3,1).reshape(-1, FN)
    self.db = np.sum(dout, axis=0)
    self.dW = np.dot(self.col.T, dout)
    self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)
    dcol = np.dot(dout, self.col_W.T)
    dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)
    return dx

In [ ]:
class Pooling:
  def __init__(self, pool_h, pool_w, stride=1, pad=0):
    self.pool_h = pool_h
    self.pool_w = pool_w
    self.stride = stride
    self.pad = pad

  def forward(self,x):
    N, C, H, W = x.shape
    out_h = int(1+(H-self.pool_h)/self.stride)
    out_w = int(1+(W-self.pool_w)/self.stride)
    # 전개(1)
    col = im2col(x,self.pooh_h,self.pool_w,self.stride,self.pad)
    col = col.reshape(-1,self.pool_h*self.pool_w)
    # 최댓값(2)
    out = np.max(col, axis=1)
    # 성형(3)
    out = out.reshape(N,out_h,out_w,C).transpose(0,3,1,2)
    return out

  def backward(self, dout):
    dout = dout.transpose(0, 2, 3, 1)
    pool_size = self.pool_h * self.pool_w
    dmax = np.zeros((dout.size, pool_size))
    dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
    dmax = dmax.reshape(dout.shape + (pool_size,)) 
    dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
    dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)
    return dx